### Import necessary libraries

In [16]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ib_insync import *
from datetime import datetime
import time
import os

SYMBOL = 'QQQ'
DATA_FILE = f'../data/{SYMBOL}_5min.csv'
DOWNLOAD_NEW_DATA = False  # Set to True to force download from IBKR

#### Data Ingestion

In [17]:
async def download_data_from_ib():
    # Start the connection to TWS or IB Gateway
    ib = IB()
    await ib.connectAsync('127.0.0.1', 7497, clientId=1)

    # Define the contract
    contract = Stock(SYMBOL, 'SMART', 'USD')

    all_data = []
    end_date = datetime.now()

    # Download 20 years data
    for i in range(1, 41):
        endDateTime = end_date.strftime('%Y%m%d %H:%M:%S')

        # Request 5-minute historical data
        bars = ib.reqHistoricalData(
            contract,
            endDateTime=endDateTime,     # data up to this timestamp
            durationStr='6 M',           # 6 months per request
            barSizeSetting='5 mins',     # timeframe 5min
            whatToShow='TRADES',         
            useRTH=True,                 # regular trading hours only
            formatDate=1
        )

        if bars:
            # Convert the response to a DataFrame
            df = util.df(bars)
            print(f"Fetched {len(df)} bars ending on {df['date'].iloc[-1]}")
            all_data.append(df)
            end_date = pd.to_datetime(df['date'].iloc[0])
        
        time.sleep(1)

    ib.disconnect()

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df = final_df.drop_duplicates(subset=['date'], keep='first')
        final_df = final_df.sort_values('date').reset_index(drop=True)
        return final_df
    
    return pd.DataFrame()

# --- Loading Logic ---
if DOWNLOAD_NEW_DATA or not os.path.exists(DATA_FILE):
    df = await download_data_from_ib()
    if not df.empty:
        # Create the 'data' folder if it doesn't exist
        os.makedirs('data', exist_ok=True)
        df.to_csv(DATA_FILE, index=False)
        print(f"Data saved to {DATA_FILE}")
else:
    print(f"Loading local data from {DATA_FILE}...")
    df = pd.read_csv(DATA_FILE)


Loading local data from ../data/QQQ_5min.csv...


##### Feature Engineering


In [24]:
# Calculate indicators using Pandas TA
df['ATR_14'] = ta.atr(df['high'], df['low'], df['close'], length=14)
df['SMA_200'] = ta.sma(df['close'], length=200)
df['WILLR_10'] = ta.willr(df['high'], df['low'], df['close'], length=10)

# Remove initial NaN values caused by the SMA200 calculation
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

df.head()

,date,open,high,low,close,volume,average,barCount,ATR_14,SMA_200,WILLR_10
0,2009-12-24 10:10:00-05:00,45.83,45.85,45.82,45.83,561900.0,45.830,1087,0.054225,45.2797,-13.888889
1,2009-12-24 10:15:00-05:00,45.82,45.85,45.82,45.85,215700.0,45.836,611,0.052494,45.2840,-10.714286
2,2009-12-24 10:20:00-05:00,45.84,45.86,45.82,45.85,828300.0,45.835,1222,0.051602,45.2884,-12.000000
3,2009-12-24 10:25:00-05:00,45.85,45.88,45.84,45.85,961200.0,45.861,1542,0.050773,45.2929,-14.285714
4,2009-12-24 10:30:00-05:00,45.84,45.85,45.80,45.81,387300.0,45.821,679,0.050718,45.2971,-33.333333


### Backtest


In [ ]:
def ibkr_commission(shares):
    total_fees = shares * 0.005
    return total_fees

def calculate_position_size(entry_price, stop_loss, account_size, risk_per_trade_pct, max_risk_dollars, leverage=4):
    """
    Calculate the number of contracts (or shares) to buy, taking into account:
    - risk per trade as a percentage,
    - leverage,
    - maximum allowed absolute loss in dollars.
    """

    # Risk per contract
    R = abs(entry_price - stop_loss)
    if R == 0 or R < 0.01:  # minimal symbolic risk to avoid division by zero
        return 0

    risk_dollars = account_size * risk_per_trade_pct
    allowed_risk = min(risk_dollars, max_risk_dollars)
    risk_based_size = allowed_risk / R
    leverage_based_size = (account_size * leverage) / entry_price
    position_size = int(min(risk_based_size, leverage_based_size))

    return position_size, R * position_size

def run_backtest(df, investment, risk_per_trade_pct, atr_multiplier, max_risk_dollars):
    equity = investment
    trades = []
    
    in_position = False
    entry_price = 0
    entry_date = None
    no_of_shares = 0
    trailing_stop_price = 0
    dollar_risk = 0
    entry_idx = 0
    fees = 0

    for i in range(1, len(df)):
        
        # --- EXIT LOGIC ---
        if in_position:
            exit_triggered = False
            exit_reason = ""

            if df['low'][i] <= trailing_stop_price:
                exit_triggered = True
                exit_reason = "TRAILING_STOP"
                exit_price = trailing_stop_price
            if df['WILLR_10'][i] > -20 and df['close'][i] < df['SMA_200'][i]:
                exit_triggered = True
                exit_reason = "WILLR_SMA"
                exit_price = df['close'][i]

            if exit_triggered:
                equity += (no_of_shares * df['close'][i])
                in_position = False
                pnl = (exit_price - entry_price) * no_of_shares - fees
                rr = pnl / dollar_risk
                trades.append({
                    "entry_date": entry_date,
                    "exit_date": df["date"][i],
                    "entry_price": entry_price,
                    "exit_price": df['close'][i],
                    "pnl": round(pnl, 2),
                    "R:R": rr,
                    "shares": no_of_shares,
                    "equity_post_trade": equity,
                    "atr_at_entry": df['ATR_14'][i],
                    "sma200_at_entry": df['SMA_200'][i],
                    "willr10_at_entry": df['WILLR_10'][i],
                    "exit_reason": exit_reason,
                    "fees": fees,
                })
                continue # Skip to the next loop iteration after closing the position

            # Calculate potential new stop and update only if higher
            potential_stop = round(df['close'][i] - (df['ATR_14'][i] * atr_multiplier), 2)
            trailing_stop_price = max(trailing_stop_price, potential_stop)
        # --- ENTRY LOGIC ---
        if not in_position:
            if df['WILLR_10'][i] < -80 and df['close'][i] > df['SMA_200'][i]:
                entry_date = df['date'][i]

                entry_price = df['open'][i+1]
                
                atr_value = df['ATR_14'][i]
                if atr_value <= 0: 
                    continue # Avoid division by zero if ATR is zero
                
                # --- POSITION SIZING BASED ON ATR AND RISK ---
                risk_per_share = atr_value * atr_multiplier
                # Set initial stop loss
                trailing_stop_price = round(entry_price - risk_per_share, 2)

                no_of_shares, dollar_risk = calculate_position_size(
                    entry_price=entry_price,
                    stop_loss=trailing_stop_price,
                    account_size=equity,
                    risk_per_trade_pct=risk_per_trade_pct,
                    max_risk_dollars=max_risk_dollars,
                    leverage=4
                )

                fees = ibkr_commission(no_of_shares) * 2
                
                # Execute trade
                equity -= (no_of_shares * entry_price) 
                
                in_position = True

    # Close position if still open at the end
    if in_position:
        equity += (no_of_shares * df.close[i])

    return pd.DataFrame(trades)
   
# Execute backtest
trades_df = run_backtest(df, 100000, risk_per_trade_pct=0.02, atr_multiplier=10, max_risk_dollars=30000)
trades_df.head()

,entry_date,exit_date,entry_price,exit_price,pnl,R:R,shares,equity_post_trade,atr_at_entry,sma200_at_entry,willr10_at_entry,exit_reason,fees
0,2009-12-24 12:20:00-05:00,2009-12-28 14:25:00-05:00,45.90,45.98,212.10,0.106061,6060,100484.80,0.034884,45.7274,-83.333333,TRAILING_STOP,30.300
1,2009-12-28 14:40:00-05:00,2009-12-30 11:45:00-05:00,45.98,46.06,396.60,0.197368,5288,100907.84,0.043665,46.0958,-9.090909,WILLR_SMA,26.440
2,2009-12-31 10:00:00-05:00,2009-12-31 12:05:00-05:00,46.15,46.06,-467.59,-0.231707,4922,100464.86,0.036420,46.0840,-18.181818,WILLR_SMA,24.610
3,2010-01-04 10:15:00-05:00,2010-01-06 13:00:00-05:00,46.29,46.27,-64.40,-0.032051,2576,100413.34,0.050948,46.3700,-7.142857,WILLR_SMA,12.880
4,2010-01-08 10:50:00-05:00,2010-01-11 10:15:00-05:00,46.22,46.25,-208.16,-0.103659,2449,100486.81,0.072197,46.2296,-75.000000,TRAILING_STOP,12.245


#### Performance analysis


In [23]:
if not trades_df.empty:
    # Preparazione dati per confronto
    # Creiamo un equity curve giornaliera per Sharpe Ratio e plot
    trades_df['exit_date'] = pd.to_datetime(trades_df['exit_date'], utc=True).dt.tz_convert('America/New_York')
    trades_df = trades_df.set_index('exit_date').sort_index()
    
    # Capitale Iniziale
    initial_cap = 100000
    
    # Buy & Hold Equity
    first_price = df['close'].iloc[0]
    bh_shares = initial_cap / first_price
    df['bh_equity'] = df['close'] * bh_shares
    
    # Strategy Equity (Ricostruita nel tempo)
    # Creiamo una serie temporale che allinea l'equity al tempo
    strategy_equity_series = pd.Series(initial_cap, index=df['date'])
    
    # Aggiorniamo l'equity nei punti in cui chiudiamo i trade
    cumulative_pnl = trades_df['pnl'].cumsum()
    
    # Plotting
    plt.figure(figsize=(18, 8))
    
    # Plot Buy & Hold
    plt.plot(df['date'], df['bh_equity'], label='Buy & Hold (QQQ)', color='gray', alpha=0.6)
    
    # Plot Strategy (Punti di uscita)
    plt.plot(trades_df.index, trades_df['equity'], label='Mean Reversion Strategy', color='blue', linewidth=2)
    
    plt.title('Equity Curve: Strategy vs Benchmark', fontsize=16)
    plt.ylabel('Equity ($)')
    plt.legend()
    plt.show()
    
    # --- Metriche Chiave ---
    total_return = (trades_df['equity'].iloc[-1] / initial_cap) - 1
    bh_return = (df['bh_equity'].iloc[-1] / initial_cap) - 1
    
    wins = trades_df[trades_df['pnl'] > 0]
    losses = trades_df[trades_df['pnl'] <= 0]
    win_rate = len(wins) / len(trades_df)
    
    # Profit Factor
    gross_profit = wins['pnl'].sum()
    gross_loss = abs(losses['pnl'].sum())
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.inf
    
    # Drawdown
    # Calcoliamo il drawdown sulla serie dei trade (semplificato)
    equity_curve = trades_df['equity']
    running_max = equity_curve.cummax()
    drawdown = (equity_curve - running_max) / running_max
    max_dd = drawdown.min()

    print("=== PERFORMANCE REPORT ===")
    print(f"Return Strategia: {total_return*100:.2f}%")
    print(f"Return Buy & Hold: {bh_return*100:.2f}%")
    print(f"Alpha (Excess Return): {(total_return - bh_return)*100:.2f}%")
    print("-" * 30)
    print(f"Trades Totali: {len(trades_df)}")
    print(f"Win Rate: {win_rate*100:.2f}%")
    print(f"Profit Factor: {profit_factor:.2f}")
    print(f"Max Drawdown: {max_dd*100:.2f}%")
    print(f"Avg Win: ${wins['pnl'].mean():.2f}")
    print(f"Avg Loss: ${losses['pnl'].mean():.2f}")

else:
    print("Nessun trade generato. Controlla i parametri o i dati.")

KeyError: 'exit_date'